# Installing the Package

In [4]:
!pip install dice-ml
!pip install RISE
import numpy as np
import timeit
import random
import dice_ml
from dice_ml.utils import helpers  # helper functions
from dice_ml import Dice
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loading Dataset

In [8]:
dataset = helpers.load_adult_income_dataset()
adult_info = helpers.get_adult_data_info()
adult_info

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

# Splitting Dataset

In [35]:
target = dataset["income"]
train_dataset, test_dataset, y_train, y_test = train_test_split(dataset,
                                                                target,
                                                                test_size=0.2,
                                                                random_state=0,
                                                                stratify=target)
x_train = train_dataset.drop('income', axis=1)
x_test = test_dataset.drop('income', axis=1)

In [36]:
x_test.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week
14946,29,Private,HS-grad,Married,Blue-Collar,White,Female,38
24228,50,Other/Unknown,Some-college,Married,Other/Unknown,White,Male,40
605,50,Private,Bachelors,Married,Professional,White,Male,40
6238,41,Private,School,Married,Blue-Collar,White,Male,30
25954,28,Other/Unknown,Assoc,Separated,Other/Unknown,White,Female,40


In [37]:
d = dice_ml.Data(dataframe=train_dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')

In [38]:
numerical = ["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical)

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model = clf.fit(x_train, y_train)

In [39]:
# Using sklearn backend
m = dice_ml.Model(model=model, backend="sklearn")
# Using method=random for generating CFs
exp = dice_ml.Dice(d, m, method="random")

In [40]:
e1 = exp.generate_counterfactuals(x_test[0:1], total_CFs=2, desired_class="opposite")
e1.visualize_as_dataframe(show_only_changes=False)

100%|██████████| 1/1 [00:00<00:00,  5.51it/s]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Bachelors,-,Service,-,-,-,1
1,-,-,Masters,-,Other/Unknown,-,-,-,1


In [23]:
# Changing only age and education
e2 = exp.generate_counterfactuals(x_test[0:1],
                                  total_CFs=2,
                                  desired_class="opposite",
                                  features_to_vary=["education", "occupation"]
                                  )
e2.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:00<00:00,  3.40it/s]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Bachelors,-,White-Collar,-,-,-,1
1,-,-,Masters,-,Sales,-,-,-,1


# More confined Restrictions

In [24]:
# Restricting age to be between [20,30] and Education to be either {'Doctorate', 'Prof-school'}.
e3 = exp.generate_counterfactuals(x_test[0:1],
                                  total_CFs=2,
                                  desired_class="opposite",
                                  permitted_range={'age': [20, 30], 'education': ['Doctorate', 'Prof-school']})
e3.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:00<00:00,  4.06it/s]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,Government,-,Single,-,-,-,-,1
1,-,-,Prof-school,-,White-Collar,-,-,-,1


In [47]:
e4 = exp.generate_counterfactuals(x_test[0:1],
                                  total_CFs=5,
                                  desired_class="opposite",
                                  permitted_range={'age': [28, 30],  'education': ["Some-college",'Doctorate', 'Prof-school']})
e4.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:00<00:00,  2.21it/s]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Doctorate,-,White-Collar,-,-,-,1
1,-,Government,Doctorate,-,-,-,-,-,1
2,-,-,Doctorate,-,-,-,Male,-,1
3,-,Self-Employed,-,-,White-Collar,-,-,-,1
4,-,Government,-,-,Professional,-,-,-,1


# Regression Example 

In [1]:
import dice_ml
from dice_ml import Dice

from sklearn.datasets import load_iris, load_boston
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

import pandas as pd

In [2]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

In [6]:
df_housing = pd.DataFrame(housing.data, columns=housing.feature_names)
df_housing['outcome_name'] = pd.Series(housing.target)
df_housing.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome_name
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [7]:
continuous_features_cali = df_housing.drop('outcome_name', axis=1).columns.tolist()
target = df_housing['outcome_name']

In [8]:
datasetX = df_housing.drop('outcome_name', axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=0)
categorical_features = x_train.columns.difference(continuous_features_cali)

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, continuous_features_cali),
        ('cat', categorical_transformer, categorical_features)])

regr_cali = Pipeline(steps=[('preprocessor', transformations),
                              ('regressor', RandomForestRegressor())])
model_cali = regr_cali.fit(x_train, y_train)

In [46]:
d_cali = dice_ml.Data(dataframe=df_housing, continuous_features=continuous_features_cali, outcome_name='outcome_name')
# We provide the type of model as a parameter (model_type)
m_cali = dice_ml.Model(model=model_cali, backend="sklearn", model_type='regressor')
exp_genetic_cali = Dice(d_cali, m_cali, method="genetic")

In [48]:
# Multiple queries can be given as input at once
query_instances_cali = x_train[17:19]
genetic_cali = exp_genetic_cali.generate_counterfactuals(query_instances_cali, 
                                                             total_CFs=2, 
                                                             desired_range=[3.5, 4])
genetic_cali.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 2/2 [00:01<00:00,  1.55it/s]

Query instance (original outcome : 3)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome_name
0,6.0145,14.0,6.156749,1.082729,1858.0,2.696662,34.23,-118.87,3.10116



Diverse Counterfactual set (new outcome: [3, 4])


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome_name
0,-,-,6.2,1.1,-,2.7,-,-,-
0,5.3558,-,6.5,1.0,1880.0,2.6,34.17,-118.9,3.0676099999999997
0,5.0519,-,5.1,1.0,1824.0,0.7,34.17,-118.9,3.0554305999999993
0,6.8645,23.0,7.3,1.0,-,3.5,33.72,-117.96,3.097220199999998


Query instance (original outcome : 2)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome_name
0,3.6413,40.0,4.604736,1.041894,1318.0,2.400729,34.27,-119.26,2.33726



Diverse Counterfactual set (new outcome: [3, 4])


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome_name
0,5.3561,29.0,5.3,1.0,-,2.6,37.27,-121.98,3.0205099999999994
0,5.4781,27.0,6.4,1.1,-,2.7,38.01,-122.53,3.02902
0,6.8642,21.0,7.3,1.0,-,3.1,37.94,-121.95,3.0543000000000005
0,3.3889,32.0,4.1,1.1,1323.0,1.8,34.03,-118.47,3.4381500999999997


# Local/Global Feature Importance 

In [ ]:
query_instance = x_test[0:1]
imp = exp.local_feature_importance(query_instance, total_CFs=10)
print(imp.local_importance)
query_instances = x_test[0:20]
imp = exp.global_feature_importance(query_instances)
print(imp.summary_importance)

# Deep Learning Application: Gradient Descent Method 

In [16]:
!pip install tensorflow
import tensorflow as tf

     |████████████████████████████████| 489.6 MB 19 kB/s               ████▏                      | 139.9 MB 3.5 MB/s eta 0:01:40
     |████████████████████████████████| 2.1 MB 84.3 MB/s            
     |████████████████████████████████| 4.1 MB 61.3 MB/s            
     |████████████████████████████████| 126 kB 71.9 MB/s            
     |████████████████████████████████| 57 kB 7.9 MB/s             
     |████████████████████████████████| 1.1 MB 52.1 MB/s            
     |████████████████████████████████| 42 kB 2.0 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 65 kB 6.3 MB/s             
     |████████████████████████████████| 79 kB 10.4 MB/s            
     |████████████████████████████████| 14.5 MB 30.2 MB/s            
     |████████████████████████████████| 1.3 MB 74.9 MB/s            
     |████████████████████████████████| 463 kB 73.5 MB/s            
     |████████████████████████████████| 5.8 MB 78.1 MB/s            
    

2022-01-30 15:56:06.879907: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-30 15:56:06.879957: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [34]:
# supress deprecation warnings from TF
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

backend = 'TF'+tf.__version__[0]  # TF1
ML_modelpath = helpers.get_adult_income_modelpath(backend=backend)
# Step 2: dice_ml.Model
m = dice_ml.Model(model_path=ML_modelpath, backend=backend)
# Step 3: initiate DiCE
exp = dice_ml.Dice(d, m)

NameError: name 'd' is not defined

In [ ]:
# query instance in the form of a dictionary or a dataframe; keys: feature name, values: feature value
query_instance = {'age': 22,
                  'workclass': 'Private',
                  'education': 'HS-grad',
                  'marital_status': 'Single',
                  'occupation': 'Service',
                  'race': 'White',
                  'gender': 'Female',
                  'hours_per_week': 45}

In [ ]:
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

In [ ]:
dice_exp.visualize_as_dataframe(show_only_changes=True)